Problem statement:-
This is a time series problem dealing with air quality

Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/train.csv")
train

In [ ]:
test = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/test.csv")
test

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv")
submission

Analyse targets

In [ ]:
#plt.plot(train.date_time, train.target_carbon_monoxide)
#plt.xlabel('Date')
#plt.ylabel('Carbon Monoxide')
#plt.title('Carbon Monoxide')
#plt.show()

In [ ]:
#plt.plot(train.date_time, train.target_benzene)
#plt.xlabel('Date')
#plt.ylabel('Benzene')
#plt.title('Benzene')
#plt.show()

In [ ]:
#plt.plot(train.date_time, train.target_nitrogen_oxides)
#plt.xlabel('Date')
#plt.ylabel('Nitrogen Oxides')
#plt.title('Nitrogen Oxides')
#plt.show()

Define target and drop from train

In [ ]:
target = train[['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]
train.drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], axis = 1, inplace = True)
train

Combine train and test

In [ ]:
combi = train.append(test)
combi

Drop date_time

In [ ]:
combi.drop(['date_time'], axis = 1, inplace = True)
combi

Check for null values

In [ ]:
combi.isnull().sum()

Normalise combi

In [ ]:
#combi = (combi.max() - combi) / (combi.max() - combi.min())
#combi

Define X and y

In [ ]:
y = target
X = combi[: len(train)]
X_test = combi[len(train) :]

Split X for training and validationg

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=1)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape

Define model

see paper on regressor chains:- https://www.scitepress.org/Papers/2019/79328/79328.pdf

In [ ]:
pip install sklearn-contrib-py-earth

In [ ]:
from sklearn.multioutput import RegressorChain
from pyearth import Earth


model = RegressorChain(Earth()).fit(X_train, y_train)
model.score(X_train, y_train)

Predict on validation set

In [ ]:
y_pred = model.predict(X_val)
model.score(X_val, y_val)

Predict on test set

In [ ]:
predictions = model.predict(X_test)
predictions.shape

In [ ]:
predictions[predictions < 0] = 0

Prepare submission

In [ ]:
submission.target_carbon_monoxide = predictions[:, 0]
submission.target_benzene = predictions[:, 1]
submission.target_nitrogen_oxides = predictions[:, 2]

In [ ]:
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission